# Ноутбук с парсингом фаз луны с сайта https://mirkosmosa.ru

In [1]:
# Загрузим необходимые библиотеки для парсинга веб страниц
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
import json
from pprint import pprint
from selenium.webdriver.firefox.options import Options
import datetime

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# Сформируем опции для запуска движка браузера
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument("--disable-dev-shm-usage");
driver = webdriver.Firefox(options=chrome_options)

Создадим функцию для формирования списка дат

In [ ]:
def get_dates_list(start_date, num_days):
    return [start_date + datetime.timedelta(days=x) for x in range(num_days)]

Распарсим вебстраницу
  
Поскольку страницы легковесные, то для удобства последующего использования загрузим данные сразу до 2030 года

In [25]:
months = ['', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
shine_dict = {
    '0': 0,
    '1': 0,
    '2': 50,
    '3': 50,
    '4': 100,
    '5': 100,
    '6': 50,
    '7': 50
}

moon_rows = []

for year in range(2013,2030):
    for month in range(1,13):
        driver.get(f'https://mirkosmosa.ru/lunar-calendar/phase-moon/{year}/{months[month]}')
        html = driver.page_source
        soup = BeautifulSoup(html)
        rows = soup.find_all(attrs = {'class': 'month_row'})
        for row in rows:
            date_str_list = row.find(attrs = {'class': 'month_cel_date'}).text.split(' ')
            phase = row.find('img')['src'].split('/')[-1].split('.')[0].split('_')
            date_str = f'{date_str_list[0]}.{month}.{year}'
            moon_shine = 0
            shine_rise_intervals =  np.round(np.linspace(0, 50, 7)[:-1])
            moon_shine = shine_dict[phase[0]]
            if len(phase) > 1:
                coef_growing = 1
                if int(phase[0]) > 4:
                    coef_growing = -1
                moon_shine += coef_growing * shine_rise_intervals[int(phase[1])]
            moon_rows.append([date_str, moon_shine])

moon_calendar = pd.DataFrame(moon_rows, columns = ['date', 'moon_shine'])

Преобразуем дату в формат pandas.datetime и отобразим получившийся DataFrame

In [26]:
moon_calendar.date = pd.to_datetime(moon_calendar.date, format = '%d.%m.%Y')
display(moon_calendar)

,date,moon_shine
0,2013-01-01,83.0
1,2013-01-02,75.0
2,2013-01-03,67.0
3,2013-01-04,58.0
4,2013-01-05,50.0
...,...,...
6816,2029-12-30,25.0
6817,2029-12-31,17.0
6818,2029-12-06,8.0
6819,2029-12-26,75.0


Отлично, DataFreme сформирован
  
Теперь избавимся от дубликатов

In [28]:
moon_calendar = moon_calendar.drop_duplicates()
moon_calendar

,date,moon_shine
0,2013-01-01,83.0
1,2013-01-02,75.0
2,2013-01-03,67.0
3,2013-01-04,58.0
4,2013-01-05,50.0
...,...,...
6813,2029-12-27,67.0
6814,2029-12-28,50.0
6815,2029-12-29,25.0
6816,2029-12-30,25.0


Сохраним полученные данные в формате csv

In [ ]:
moon_calendar.to_csv('moon.csv', sep = ';', index = False)